In [77]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Polybench Experimental Results

This notebook puts together how we run Polybench experiments using the `pb-flow` script, and how we process the run results to get the formatted table.

## Setup

We need to put the `python/` directory into the `PYTHONPATH`. 

In [71]:
import sys
import os
import pandas as pd

sys.path.append(os.path.dirname(os.path.realpath(os.getcwd())))

In [72]:
import python.utils.polybench as pb_utils

In [73]:
len(pb_utils.discover_examples('./example/polybench'))

30

In [80]:
options = {
  "baseline": pb_utils.PbFlowOptions(
    pb_dir=os.path.join(pb_utils.get_project_root(), 'example', 'polybench'),
    work_dir=os.path.join(pb_utils.get_project_root(), 'tmp', 'phism', 'pb-flow.baseline'),
    job=30,
    polymer=False,
    cosim=True,
    debug=True,
    dataset='MINI',
    cleanup=False,
    loop_transforms=False,
  ),
  "polymer": pb_utils.PbFlowOptions(
    pb_dir=os.path.join(pb_utils.get_project_root(), 'example', 'polybench'),
    work_dir=os.path.join(pb_utils.get_project_root(), 'tmp', 'phism', 'pb-flow.polymer'),
    job=30,
    polymer=True,
    cosim=True,
    debug=True,
    dataset='MINI',
    cleanup=False,
    loop_transforms=True,
  ),
}

## Run `pb-flow`

In [82]:
pb_utils.pb_flow_runner(options["baseline"])

>>> Starting 30 jobs (work_dir=/mnt/ccnas2/bdp/rz3515/projects/phism/tmp/phism/pb-flow.baseline) ...
>>> Finished nussinov        elapsed: 126.044907 secs   Status: 0  Error: "No Error"
>>> Finished trmm            elapsed: 142.022824 secs   Status: 1  Error: "Phism syn failed."
>>> Finished floyd-warshall  elapsed: 159.610061 secs   Status: 0  Error: "No Error"
>>> Finished gemver          elapsed: 220.636748 secs   Status: 1  Error: "Cosim failed."
>>> Finished symm            elapsed: 223.885276 secs   Status: 1  Error: "Cosim failed."
>>> Finished jacobi-1d       elapsed: 229.468748 secs   Status: 0  Error: "No Error"
>>> Finished syrk            elapsed: 232.627230 secs   Status: 1  Error: "Cosim failed."
>>> Finished atax            elapsed: 236.069195 secs   Status: 0  Error: "No Error"
>>> Finished bicg            elapsed: 238.165721 secs   Status: 0  Error: "No Error"
>>> Finished mvt             elapsed: 242.202244 secs   Status: 0  Error: "No Error"
>>> Finished syr2k       

In [83]:
pb_utils.pb_flow_runner(options["polymer"])

>>> Starting 30 jobs (work_dir=/mnt/ccnas2/bdp/rz3515/projects/phism/tmp/phism/pb-flow.polymer) ...
>>> Finished floyd-warshall  elapsed: 129.300979 secs   Status: 0  Error: "No Error"
>>> Finished nussinov        elapsed: 130.805650 secs   Status: 0  Error: "No Error"
>>> Finished symm            elapsed: 161.053921 secs   Status: 1  Error: "Phism syn failed."
>>> Finished covariance      elapsed: 182.277354 secs   Status: 1  Error: "Phism syn failed."
>>> Finished atax            elapsed: 202.116114 secs   Status: 1  Error: "Cosim failed."
>>> Finished gemver          elapsed: 223.477063 secs   Status: 1  Error: "Cosim failed."
>>> Finished ludcmp          elapsed: 227.788396 secs   Status: 1  Error: "Phism syn failed."
>>> Finished jacobi-1d       elapsed: 230.364197 secs   Status: 0  Error: "No Error"
>>> Finished syrk            elapsed: 230.477402 secs   Status: 1  Error: "Cosim failed."
>>> Finished bicg            elapsed: 237.728273 secs   Status: 0  Error: "No Error"
>>> Fini

## Results

Here we list all the results as a table.

In [87]:
baseline = pb_utils.to_pandas(pb_utils.process_pb_flow_result_dir(options['baseline'].work_dir, options['baseline']))
baseline

,name,phism_synth,tbgen_cosim,phism_cosim,latency,DSP_usage,FF_usage,LUT_usage,BRAM_18K_usage,URAM_usage,DSP_avail,FF_avail,LUT_avail,BRAM_18K_avail,URAM_avail
0,2mm,SUCCESS,SUCCESS,SUCCESS,92642,14,2584,2985,0,0,220,106400,53200,280,0
1,3mm,SUCCESS,SUCCESS,SUCCESS,143531,16,3322,3499,0,0,220,106400,53200,280,0
2,adi,SUCCESS,SUCCESS,SUCCESS,782432,34,6748,9026,0,0,220,106400,53200,280,0
3,atax,SUCCESS,SUCCESS,SUCCESS,12243,14,1641,2204,0,0,220,106400,53200,280,0
4,bicg,SUCCESS,SUCCESS,SUCCESS,10163,14,1745,2125,0,0,220,106400,53200,280,0
5,cholesky,SUCCESS,SUCCESS,SUCCESS,277039,20,15169,13431,0,0,220,106400,53200,280,0
6,correlation,SUCCESS,SUCCESS,SUCCESS,126261,15,3406,4256,0,0,220,106400,53200,280,0
7,covariance,SUCCESS,SUCCESS,SUCCESS,109464,14,2685,3282,0,0,220,106400,53200,280,0
8,deriche,CANNOT_SYNTH,SUCCESS,NO_LOG,237664,None,None,None,None,None,None,None,None,None,None
9,doitgen,SUCCESS,SUCCESS,SUCCESS,81041,14,1704,2395,0,0,220,106400,53200,280,0


In [86]:
polymer = pb_utils.to_pandas(pb_utils.process_pb_flow_result_dir(options['polymer'].work_dir, options['polymer']))
polymer

,name,phism_synth,tbgen_cosim,phism_cosim,latency,DSP_usage,FF_usage,LUT_usage,BRAM_18K_usage,URAM_usage,DSP_avail,FF_avail,LUT_avail,BRAM_18K_avail,URAM_avail
0,2mm,SUCCESS,SUCCESS,SUCCESS,108895,54,17926,16708,0,0,220,106400,53200,280,0
1,3mm,SUCCESS,SUCCESS,SUCCESS,173643,84,24745,25758,0,0,220,106400,53200,280,0
2,adi,SUCCESS,SUCCESS,NO_LOG,768392,None,None,None,None,None,None,None,None,None,None
3,atax,SUCCESS,SUCCESS,COSIM_FAILED,None,20,5870,6664,0,0,220,106400,53200,280,0
4,bicg,SUCCESS,SUCCESS,SUCCESS,23798,20,5896,6660,0,0,220,106400,53200,280,0
5,cholesky,SUCCESS,SUCCESS,SUCCESS,124921,19,5856,8081,0,0,220,106400,53200,280,0
6,correlation,SUCCESS,SUCCESS,COSIM_FAILED,None,29,14710,16308,0,0,220,106400,53200,280,0
7,covariance,SUCCESS,SUCCESS,NO_LOG,103199,None,None,None,None,None,None,None,None,None,None
8,deriche,SUCCESS,SUCCESS,NO_LOG,237664,None,None,None,None,None,None,None,None,None,None
9,doitgen,SUCCESS,SUCCESS,SUCCESS,96242,16,3874,4697,0,0,220,106400,53200,280,0


In [88]:
base_succ = pb_utils.filter_success(baseline)
poly_succ = pb_utils.filter_success(polymer)

names = sorted(list(set(base_succ['name']) & set(poly_succ['name'])))

df = pd.DataFrame(columns=('name', 'base_lat', 'poly_lat'),
                  data=(zip(names,
                            base_succ[base_succ['name'].isin(names)]['latency'],
                            poly_succ[poly_succ['name'].isin(names)]['latency'])),
                  dtype=object)
df['speedup'] = (df['base_lat'] / df['poly_lat'])
df

,name,base_lat,poly_lat,speedup
0,2mm,92642,108895,0.850746
1,3mm,143531,173643,0.826587
2,bicg,10163,23798,0.427053
3,cholesky,277039,124921,2.217714
4,doitgen,81041,96242,0.842054
5,durbin,5462,9847,0.554687
6,floyd-warshall,432081,434885,0.993552
7,gesummv,6151,12092,0.508683
8,jacobi-1d,3141,24872,0.126287
9,jacobi-2d,134461,946620,0.142043
